In [ ]:
!alienv enter AliPhysics/latest

Currently Loaded Modulefiles:
  1) BASE/1.0                        8) Vc/1.4.1-1
  2) AliEn-Runtime/v2-19-le-2        9) AliEn-ROOT-Legacy/0.1.1-1
  3) Python-modules/1.0-2           10) AliRoot/v5-09-50-1
  4) ROOT/v6-16-00-2                11) RooUnfold/V02-00-01-alice4-2
  5) boost/v1.68.0-1                12) treelite/2a12742269-1
  6) cgal/v4.6.3-1                  13) KFParticle/v1.1-1
  7) fastjet/v3.2.1_1.024-alice3-1  14) AliPhysics/latest
Use alienv list to list loaded modules. Use exit to exit this environment.
[AliPhysics/latest] ~/cernbox/postdoc/DHFeCorrLocal/dhfcorr/qa $> 

In [ ]:
import seaborn as sns
import ROOT

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
print('a')

In [ ]:
path_to_root_file = '/data2/data/d_hfe/root/D0_HMV0/'
configuration_name = 'D0'
folder_name = 'DHFeCorrelation_' + configuration_name

In [ ]:
import glob

In [ ]:
files = glob.glob(path_to_root_file+"*.root")
print(files[0])

In [ ]:
import io

In [ ]:
print(conversor_16)

In [ ]:
conversor_16 = io.StringIO("Child Number	Period\n\
child 1	FILTER_p-p_208_LHC16d\n \
child 2	FILTER_p-p_208_LHC16e\n \
child 3	FILTER_p-p_208_LHC16g\n \
child 4	FILTER_p-p_208_LHC16h\n \
child 5	FILTER_p-p_208_LHC16j\n \
child 6	FILTER_p-p_208_LHC16k\n \
child 7	FILTER_p-p_208_LHC16l\n \
child 8	FILTER_p-p_208_LHC16o\n \
child 9	FILTER_p-p_208_LHC16p\n ")

conversor_17 = io.StringIO("Child Number	Period\n\
child 1	FILTER_p-p_208_LHC17c\n\
child 2	FILTER_p-p_208_LHC17e\n\
child 3	FILTER_p-p_208_LHC17f\n\
child 4	FILTER_p-p_212_LHC17g\n\
child 5	FILTER_p-p_208_LHC17h\n\
child 6	FILTER_p-p_208_LHC17i\n\
child 7	FILTER_p-p_208_LHC17j\n\
child 8	FILTER_p-p_208_LHC17k\n\
child 9	FILTER_p-p_208_LHC17l\n\
child 10	FILTER_p-p_208_LHC17m\n\
child 11	FILTER_p-p_208_LHC17o\n\
child 12	FILTER_p-p_208_LHC17r\n")

conversor_18 = io.StringIO("Child Number	Period\n\
child 1	FILTER_p-p_208_LHC18b\n\
child 2	FILTER_p-p_208_LHC18d\n\
child 3	FILTER_p-p_208_LHC18e\n\
child 4	FILTER_p-p_208_LHC18f\n\
child 5	FILTER_p-p_208_LHC18g\n\
child 6	FILTER_p-p_208_LHC18h\n\
child 7	FILTER_p-p_208_LHC18i\n\
child 8	FILTER_p-p_208_LHC18k\n\
child 9	FILTER_p-p_208_LHC18l\n\
child 10	FILTER_p-p_208_LHC18m\n\
child 11	FILTER_p-p_208_LHC18n\n\
child 12	FILTER_p-p_208_LHC18o\n\
child 13	FILTER_p-p_208_LHC18p\n")
        
table_convertsion = pd.concat([pd.read_csv(conversor_16, delimiter='	'), 
                              pd.read_csv(conversor_17, delimiter='	'),
                            pd.read_csv(conversor_18, delimiter='	')],ignore_index=True)
table_convertsion['Period'] = table_convertsion['Period'].apply(lambda x : x.split('LHC')[1])
table_convertsion['Child Number'] = table_convertsion['Child Number'].apply(lambda x: int(x.split('child ')[1]))
table_convertsion['Year'] = table_convertsion['Period'].apply(lambda x : x[:2])
table_convertsion.set_index(['Year','Child Number'],inplace=True)

In [ ]:
n_events_with_trigger = list()
n_events_with_DHFe = list()

for f in files:
    file_root = ROOT.TFile.Open(f)
    file_root.cd('DHFeCorrelation_D0')
    list_event_qa = ROOT.gDirectory.Get('EventsQA')
    
    events_with_trigger = list_event_qa.FindObject('VertexZ_before')
    events_with_trigger.GetXaxis().SetRangeUser(-10.,10.)
    n_events = events_with_trigger.Integral()
    n_events_with_trigger.append(n_events)
    
    events_with_DHFe = list_event_qa.FindObject('VertexZ_after')
    events_with_DHFe.GetXaxis().SetRangeUser(-10.,10.)
    n_events_DHFe = events_with_DHFe.Integral()
    n_events_with_DHFe.append(n_events_DHFe)

In [ ]:
year = [x.split('/')[-1].split('_')[0] for x in files]
child_number = [int(('child' + x.split('/')[-1].split('child')[1]).split('.')[0].split('_')[1]) for x in files]
event_counts = pd.DataFrame([year, child_number, n_events_with_trigger, n_events_with_DHFe]).T
event_counts.columns = ['Year', 'Child Number', 'Number of Events', 'Events with D and e']

In [ ]:
event_counts = event_counts.groupby(['Year', 'Child Number']).sum().join(table_convertsion).reset_index()

In [ ]:
sns.set_context('notebook')
ax = event_counts.plot.bar(x='Period', y='Number of Events',
                           color='red', label='Number of Events with HMV0 Trigger')
ax.set_ylabel('Frequency')

ax.get_figure().savefig('All_event_with_trigger.pdf', bbox_inches='tight')

In [ ]:
ax = event_counts.plot.bar(x='Period', y='Events with D and e', 
                           color='red', label='Number of Events with D and e')
ax.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
ax.set_ylabel('Frequency')
ax.get_figure().savefig('All_event_with_De.pdf', bbox_inches='tight')

In [ ]:
event_counts['Events with D and e'] = event_counts['Events with D and e'].astype(np.float32)
event_counts['Number of Events'] = event_counts['Number of Events'].astype(np.float32)
event_counts['ratio'] = event_counts['Events with D and e']/event_counts['Number of Events']

event_counts['ratio_error'] = event_counts['ratio']* np.sqrt( 
    (np.sqrt(event_counts['Events with D and e'])/event_counts['Events with D and e'])** 2  +
    (np.sqrt(event_counts['Number of Events'])/event_counts['Number of Events'])** 2)

In [ ]:
sns.set_context('notebook')
ax = event_counts.plot.bar(x='Period', y='ratio', color='red', label = 'Selected/Total')
ax.set_ylim(1.1*np.array(ax.get_ylim()))
ax.set_ylabel('Ratio')
ax.get_figure().savefig('ratio.pdf', bbox_inches='tight')

In [ ]:
print('Events with D meson and electrons')
print('{0:.1f} M events'.format(event_counts['Events with D and e'].sum()/(10**6)))

In [ ]:
print('All Events')
print('{0:.1f} M events'.format(event_counts['Number of Events'].sum()/(10**6)))

In [ ]:
import pandas as pd

In [ ]:
df_d = pd.read_parquet('/Users/hzanoli/cernbox/postdoc/DHFeCorrLocal/selected_ml.parquet')

In [ ]:
df_d = df_d[df_d['Pt'] < 24.]
name = 'cut_bdt'
cuts_pt = pd.read_pickle('/Users/hzanoli/cernbox/postdoc/DHFeCorrLocal/'+name + '.pkl')

In [ ]:
df_d['Cut'] = cuts_pt.loc[df_d['Pt']].reset_index(drop=True)

In [ ]:
df_d = df_d[df_d['prediction'] >= df_d['Cut']]

In [ ]:
len(df_d)

In [ ]:
df_d.columns

In [ ]:
n_d = df_d.groupby(by=['GridPID', 'EventNumber', 'ID']).count()['IsParticleCandidate']

In [ ]:
importan_values = df_d[['EventNumber', 'GridPID','Pt', 'Phi', 'ID', 'IsParticleCandidate']]
importan_values.info(memory_usage='deep')

In [ ]:
d_corr = importan_values.merge(importan_values, on=['EventNumber', 'GridPID'])

In [ ]:
d_corr = d_corr[d_corr['ID_x'] != d_corr['ID_y']].copy()

In [ ]:
d_corr['IsParticleCandidate_x'] = d_corr['IsParticleCandidate_x'].astype('bool')
d_corr['IsParticleCandidate_y'] = d_corr['IsParticleCandidate_y'].astype('bool')

In [ ]:
d_corr = d_corr[((d_corr['IsParticleCandidate_x'] & ~d_corr['IsParticleCandidate_y']) | 
 (d_corr['IsParticleCandidate_y'] & ~d_corr['IsParticleCandidate_x'])) ].copy()

In [ ]:
d_corr['DeltaPhi']= d_corr['Phi_x'] - d_corr['Phi_y']

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
d_corr['DeltaPhi'].plot.hist(bins=20)